In [1]:
# Importing all the required packages
import arcpy
from arcpy.ia import *
import os, sys
# Check perform to see if Image Analyst is available
arcpy.CheckOutExtension("ImageAnalyst")
# Setting processor type and parallel processing factor
# arcpy.env.processorType = "GPU"
# # Parallel Processing
# arcpy.env.parallelProcessingFactor = "50%"

'CheckedOut'

In [2]:
# # Applying Histogram Equalization on the loaded image
# RGB_imagery = r"E:\Nitin.Rai\ASABE2021\Dataset\ClippedRaster.tif"
# arcpy.env.workspace = RGB_imagery
# histogram_equalization = arcpy.ia.Stretch(RGB_imagery, "Histogram", None, 
#                                           None, None, None, True, 
#                                           None, None, None, True, None)
# # histogram_equalization.save(r"D:/ArcGIS Projects/Dataset/SunflowerCorrected.tif")
# # print("Histogram Equalization has been applied on the image!")
# brightness_correction = arcpy.ia.ContrastBrightness(histogram_equalization, 100, 70)
# brightness_correction.save(r"D:/ArcGIS Projects/Dataset/Brightness_Corrected2.tif")
# print("Brightness Correction was applied successfully!")

In [3]:
RGB_imagery = r"E:\Nitin.Rai\ASABE2021\Dataset\ClippedRaster.tif"
arcpy.env.workspace = RGB_imagery
bands = [Raster(os.path.join(RGB_imagery, b))
         for b in arcpy.ListRasters()]

In [4]:
path = os.path.dirname(RGB_imagery)
base1 = os.path.basename(RGB_imagery)
base = os.path.splitext(base1)[0]
ExGreen_filename = base + "_ExcessGreen.tif"
total_bands = (Float(bands[0] + bands[1] + bands[2]))
red_band = (Float(bands[0]) / total_bands)
green_band = (Float(bands[1]) / total_bands)
blue_band = (Float(bands[2]) / total_bands)
ExGreen = (2 * green_band - red_band - blue_band)
ExGreen.save(ExGreen_filename)
arcpy.env.workspace = path
print ("You have successfully exported the Excess Green.tif file")
arcpy.BuildPyramids_management(ExGreen_filename, "", "NONE", "BILINEAR","", "", "")
print ("Excess Green pyramids successfully calculated")

You have successfully exported the Excess Green.tif file
Excess Green pyramids successfully calculated


In [5]:
# Sharpening the image so the objects (crops & weeds) details are highlighted
sharpen_ExGreen = arcpy.ia.Convolution(RGB_imagery, 20)
sharpen_ExGreen.save(r"E:\Nitin.Rai\ASABE2021\Dataset\SharpenedRaster.tif")
print("You have successfully sharpened the Excess Green.tif file!")

You have successfully sharpened the Excess Green.tif file!


In [6]:
binary_raster = arcpy.ia.Threshold(ExGreen)
binary_raster.save(r"E:\Nitin.Rai\ASABE2021\Dataset\BinaryRaster.tif")
print("You have successfully perfromed Imagery Thresholding!")

You have successfully perfromed Imagery Thresholding!


In [7]:
arcpy.env.workspace = r"E:\Nitin.Rai\ASABE2021\Dataset"
inRaster = "BinaryRaster.tif"
outPolygons = r"E:\Nitin.Rai\ASABE2021\Dataset\RasterToPolygon.tif"
field = "VALUE"
arcpy.RasterToPolygon_conversion(inRaster, outPolygons, "NO_SIMPLIFY", field)
print("You have successfully converted Raster to Polygon!")

You have successfully converted Raster to Polygon!


In [8]:
# Selecting all the 1s so that we can export a shapefile for all the objects labeled 1s within the imagery
arcpy.env.workspace = r"E:\Nitin.Rai\ASABE2021\Dataset"
polygonAttached = "RasterToPolygon.shp"
selectedAttributes = arcpy.SelectLayerByAttribute_management(polygonAttached, "NEW_SELECTION", '"gridcode" > 0')
arcpy.CopyFeatures_management(selectedAttributes, 'CropPolygons')

<Result 'E:\\Nitin.Rai\\ASABE2021\\Dataset\\CropPolygons.shp'>